In [4]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Función para cargar y procesar imágenes de entrenamiento
def load_and_preprocess_images(data, img_size=(48, 48), path_col='path', label_col='label', filter_labels=None):
    images = []
    labels = []
    for index, row in data.iterrows():
        if filter_labels is None or row[label_col] in filter_labels:
            img_path = row[path_col].replace('..', 'data').replace('\\', '/')
            img = load_img(img_path, target_size=img_size, color_mode='grayscale')
            img_array = img_to_array(img)
            images.append(img_array)
            labels.append(row[label_col])
    images = np.array(images, dtype='float32') / 255.0
    return images, labels

In [6]:
# Función para cargar y procesar imágenes sin etiquetas (para el conjunto de prueba)
def load_and_preprocess_images2(data, img_size=(48, 48)):
    images = []
    for index, row in data.iterrows():
        img_path = f"data/data/images/test/{row['id_img']}.jpg"
        img = load_img(img_path, target_size=img_size, color_mode='grayscale')
        img_array = img_to_array(img)
        images.append(img_array)
    images = np.array(images, dtype='float32') / 255.0
    return images

In [7]:
# Cargar los datos
train_data = pd.read_csv('data/train_set.csv')
test_data = pd.read_csv('test_set.csv')
sample_submission = pd.read_csv('sample_submision.csv')

In [8]:
# Ajustar los nombres de las columnas según sea necesario
path_col = 'path'  # Cambia esto si el nombre de la columna es diferente
label_col = 'label'

In [9]:
# Mostrar las columnas de los DataFrames
print("Columnas de train_set.csv:", train_data.columns)
print("Columnas de test_set.csv:", test_data.columns)

Columnas de train_set.csv: Index(['id_img', 'path', 'label'], dtype='object')
Columnas de test_set.csv: Index(['id_img'], dtype='object')


In [10]:
# Cargar y procesar imágenes de entrenamiento
train_images, train_labels = load_and_preprocess_images(train_data, path_col=path_col, label_col=label_col, filter_labels=['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise'])

In [15]:
# Codificar etiquetas para 7 clases
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
train_labels_categorical = to_categorical(train_labels_encoded, num_classes=7)

In [16]:
# Dividir los datos en conjunto de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels_categorical, test_size=0.2, random_state=42)

In [45]:
def create_model(model_type='simple_cnn'):
    if model_type == 'simple_cnn':
        model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
            BatchNormalization(),
            MaxPooling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu'),
            BatchNormalization(),
            MaxPooling2D((2, 2)),
            Conv2D(128, (3, 3), activation='relu'),
            BatchNormalization(),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
            Dropout(0.5),
            Dense(7, activation='softmax')  # Ajuste para 7 clases de emociones
        ])
    elif model_type == 'deeper_cnn':
        model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
            BatchNormalization(),
            Conv2D(32, (3, 3), activation='relu'),
            BatchNormalization(),
            MaxPooling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu'),
            BatchNormalization(),
            Conv2D(64, (3, 3), activation='relu'),
            BatchNormalization(),
            MaxPooling2D((2, 2)),
            Conv2D(128, (3, 3), activation='relu'),
            BatchNormalization(),
            Conv2D(128, (3, 3), activation='relu'),
            BatchNormalization(),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
            Dropout(0.5),
            Dense(7, activation='softmax')  # Ajuste para 7 clases de emociones
        ])
    elif model_type == 'deep_cnn':
        model = Sequential([
            Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)),
            BatchNormalization(),
            Conv2D(64, (3, 3), activation='relu'),
            BatchNormalization(),
            MaxPooling2D((2, 2)),
            Conv2D(128, (3, 3), activation='relu'),
            BatchNormalization(),
            Conv2D(128, (3, 3), activation='relu'),
            BatchNormalization(),
            MaxPooling2D((2, 2)),
            Conv2D(256, (3, 3), activation='relu'),
            BatchNormalization(),
            Conv2D(256, (3, 3), activation='relu'),
            BatchNormalization(),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
            Dropout(0.5),
            Dense(7, activation='softmax')  # Ajuste para 7 clases de emociones
        ])
    elif model_type == 'very_deep_cnn':
        model = Sequential([
            Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)),
            BatchNormalization(),
            Conv2D(64, (3, 3), activation='relu'),
            BatchNormalization(),
            Conv2D(64, (3, 3), activation='relu'),
            BatchNormalization(),
            MaxPooling2D((2, 2)),
            Conv2D(128, (3, 3), activation='relu'),
            BatchNormalization(),
            Conv2D(128, (3, 3), activation='relu'),
            BatchNormalization(),
            MaxPooling2D((2, 2)),
            Conv2D(256, (3, 3), activation='relu'),
            BatchNormalization(),
            Conv2D(256, (3, 3), activation='relu'),
            BatchNormalization(),
            Conv2D(256, (3, 3), activation='relu'),
            BatchNormalization(),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
            Dropout(0.5),
            Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
            Dropout(0.5),
            Dense(7, activation='softmax')  # Ajuste para 7 clases de emociones
        ])
    elif model_type == 'vgg16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=(94, 94, 3))
        model = Sequential([
            tf.keras.layers.UpSampling2D((2, 2), input_shape=(48, 48, 1)),
            Conv2D(3, (3, 3), activation='relu'),
            base_model,
            GlobalAveragePooling2D(),
            Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
            Dropout(0.5),
            Dense(7, activation='softmax')
        ])
        for layer in base_model.layers:
            layer.trainable = False
    return model


In [34]:
# Aplicar aumento de datos avanzado
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(X_train)

In [41]:
# Parámetros para probar
model_types = ['vgg16']
epochs_list = [20, 30]
batch_sizes = [32, 64]
# 'simple_cnn', 'deeper_cnn', 'deep_cnn', 'very_deep_cnn','vgg16'
best_accuracy = 0
best_params = {}

In [42]:
# Configurar EarlyStopping y ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True, mode='max')

In [46]:
# Entrenar y evaluar cada combinación de parámetros
for model_type in model_types:
    for epochs in epochs_list:
        for batch_size in batch_sizes:
            print(f"Entrenando modelo: {model_type}, epochs: {epochs}, batch_size: {batch_size}")
            model = create_model(model_type=model_type)
            model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
            history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size), epochs=epochs, validation_data=(X_val, y_val), callbacks=[early_stopping, model_checkpoint], verbose=1)
            val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
            print(f"Validation accuracy: {val_accuracy:.4f}")
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                best_params = {'model_type': model_type, 'epochs': epochs, 'batch_size': batch_size}
                best_model = model

Entrenando modelo: vgg16, epochs: 20, batch_size: 32
Epoch 1/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 2514s 3s/step - accuracy: 0.2257 - loss: 2.6446 - val_accuracy: 0.3082 - val_loss: 1.7632
Epoch 2/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 1949s 3s/step - accuracy: 0.2674 - loss: 1.8024 - val_accuracy: 0.3297 - val_loss: 1.7325
Epoch 3/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 2192s 3s/step - accuracy: 0.2852 - loss: 1.7767 - val_accuracy: 0.3488 - val_loss: 1.7094
Epoch 4/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 1938s 3s/step - accuracy: 0.2942 - loss: 1.7720 - val_accuracy: 0.3367 - val_loss: 1.7103
Epoch 5/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 1998s 3s/step - accuracy: 0.2974 - loss: 1.7577 - val_accuracy: 0.3377 - val_loss: 1.7056
Epoch 6/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 1950s 3s/step - accuracy: 0.3062 - loss: 1.7556 - val_accuracy: 0.3329 - val_loss: 1.7145
Epoch 7/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 2001s 3s/step - accuracy: 0.2999 - loss: 1.7536 - val_accuracy: 0.3495 - val_loss: 1.6799
Epoch 8/20
721/721 ━━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [40]:
# Entrenar y evaluar cada combinación de parámetros
for model_type in model_types:
    for epochs in epochs_list:
        for batch_size in batch_sizes:
            print(f"Entrenando modelo: {model_type}, epochs: {epochs}, batch_size: {batch_size}")
            model = create_model(model_type=model_type)
            model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
            history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size), epochs=epochs, validation_data=(X_val, y_val), callbacks=[early_stopping, model_checkpoint], verbose=1)
            val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
            print(f"Validation accuracy: {val_accuracy:.4f}")
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                best_params = {'model_type': model_type, 'epochs': epochs, 'batch_size': batch_size}
                best_model = model

Entrenando modelo: simple_cnn, epochs: 20, batch_size: 32
Epoch 1/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 84s 108ms/step - accuracy: 0.2127 - loss: 3.5680 - val_accuracy: 0.2638 - val_loss: 1.8844
Epoch 2/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 64s 89ms/step - accuracy: 0.2544 - loss: 1.9021 - val_accuracy: 0.3107 - val_loss: 1.7396
Epoch 3/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 85s 118ms/step - accuracy: 0.2652 - loss: 1.8059 - val_accuracy: 0.3311 - val_loss: 1.7953
Epoch 4/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 68s 94ms/step - accuracy: 0.2932 - loss: 1.7778 - val_accuracy: 0.3405 - val_loss: 1.6843
Epoch 5/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 75s 104ms/step - accuracy: 0.3111 - loss: 1.7313 - val_accuracy: 0.4064 - val_loss: 1.5935
Epoch 6/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 68s 94ms/step - accuracy: 0.3514 - loss: 1.6927 - val_accuracy: 0.4064 - val_loss: 1.6043
Epoch 7/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 68s 94ms/step - accuracy: 0.3724 - loss: 1.6493 - val_accuracy: 0.4153 - val_loss: 1.5353
Epoch 8/20
721/721 ━━━━━━━━━━

KeyboardInterrupt: 

Entrenando modelo: simple_cnn, epochs: 20, batch_size: 32
Epoch 1/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 84s 108ms/step - accuracy: 0.2127 - loss: 3.5680 - val_accuracy: 0.2638 - val_loss: 1.8844
Epoch 2/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 64s 89ms/step - accuracy: 0.2544 - loss: 1.9021 - val_accuracy: 0.3107 - val_loss: 1.7396
Epoch 3/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 85s 118ms/step - accuracy: 0.2652 - loss: 1.8059 - val_accuracy: 0.3311 - val_loss: 1.7953
Epoch 4/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 68s 94ms/step - accuracy: 0.2932 - loss: 1.7778 - val_accuracy: 0.3405 - val_loss: 1.6843
Epoch 5/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 75s 104ms/step - accuracy: 0.3111 - loss: 1.7313 - val_accuracy: 0.4064 - val_loss: 1.5935
Epoch 6/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 68s 94ms/step - accuracy: 0.3514 - loss: 1.6927 - val_accuracy: 0.4064 - val_loss: 1.6043
Epoch 7/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 68s 94ms/step - accuracy: 0.3724 - loss: 1.6493 - val_accuracy: 0.4153 - val_loss: 1.5353
Epoch 8/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 68s 95ms/step - accuracy: 0.3788 - loss: 1.6295 - val_accuracy: 0.4487 - val_loss: 1.5049
Epoch 9/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 76s 105ms/step - accuracy: 0.3846 - loss: 1.6281 - val_accuracy: 0.4229 - val_loss: 1.5248
Epoch 10/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 73s 101ms/step - accuracy: 0.3986 - loss: 1.6011 - val_accuracy: 0.4493 - val_loss: 1.5084
Epoch 11/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 77s 106ms/step - accuracy: 0.4060 - loss: 1.5854 - val_accuracy: 0.4598 - val_loss: 1.4905
Epoch 12/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 80s 110ms/step - accuracy: 0.4053 - loss: 1.5930 - val_accuracy: 0.4626 - val_loss: 1.4613
Epoch 13/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 77s 107ms/step - accuracy: 0.4082 - loss: 1.5842 - val_accuracy: 0.4701 - val_loss: 1.4701
Epoch 14/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 66s 92ms/step - accuracy: 0.4205 - loss: 1.5564 - val_accuracy: 0.4793 - val_loss: 1.4105
Epoch 15/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 67s 93ms/step - accuracy: 0.4242 - loss: 1.5629 - val_accuracy: 0.4970 - val_loss: 1.4016
Epoch 16/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 74s 103ms/step - accuracy: 0.4301 - loss: 1.5542 - val_accuracy: 0.4956 - val_loss: 1.3946
Epoch 17/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 68s 94ms/step - accuracy: 0.4284 - loss: 1.5473 - val_accuracy: 0.4848 - val_loss: 1.4290
Epoch 18/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 68s 94ms/step - accuracy: 0.4314 - loss: 1.5418 - val_accuracy: 0.5027 - val_loss: 1.3937
Epoch 19/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 66s 91ms/step - accuracy: 0.4280 - loss: 1.5335 - val_accuracy: 0.4945 - val_loss: 1.4242
Epoch 20/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 67s 92ms/step - accuracy: 0.4370 - loss: 1.5360 - val_accuracy: 0.5056 - val_loss: 1.3689
Validation accuracy: 0.5056
Entrenando modelo: simple_cnn, epochs: 20, batch_size: 64
Epoch 1/20
361/361 ━━━━━━━━━━━━━━━━━━━━ 84s 220ms/step - accuracy: 0.2156 - loss: 3.8282 - val_accuracy: 0.2654 - val_loss: 2.0993
Epoch 2/20
361/361 ━━━━━━━━━━━━━━━━━━━━ 69s 189ms/step - accuracy: 0.2658 - loss: 2.0028 - val_accuracy: 0.3625 - val_loss: 1.7097
Epoch 3/20
361/361 ━━━━━━━━━━━━━━━━━━━━ 67s 184ms/step - accuracy: 0.2920 - loss: 1.8222 - val_accuracy: 0.3252 - val_loss: 1.7877
Validation accuracy: 0.2654
Entrenando modelo: simple_cnn, epochs: 30, batch_size: 32
Epoch 1/30
721/721 ━━━━━━━━━━━━━━━━━━━━ 75s 99ms/step - accuracy: 0.2175 - loss: 3.5563 - val_accuracy: 0.2614 - val_loss: 1.9020
Epoch 2/30
721/721 ━━━━━━━━━━━━━━━━━━━━ 71s 98ms/step - accuracy: 0.2507 - loss: 1.9201 - val_accuracy: 0.3063 - val_loss: 1.8277
Epoch 3/30
721/721 ━━━━━━━━━━━━━━━━━━━━ 67s 92ms/step - accuracy: 0.2697 - loss: 1.8419 - val_accuracy: 0.3160 - val_loss: 1.7372
Validation accuracy: 0.2614
Entrenando modelo: simple_cnn, epochs: 30, batch_size: 64
Epoch 1/30
361/361 ━━━━━━━━━━━━━━━━━━━━ 65s 171ms/step - accuracy: 0.2135 - loss: 3.8208 - val_accuracy: 0.2628 - val_loss: 2.0784
Epoch 2/30
361/361 ━━━━━━━━━━━━━━━━━━━━ 63s 174ms/step - accuracy: 0.2488 - loss: 2.0052 - val_accuracy: 0.3089 - val_loss: 1.8089
Epoch 3/30
361/361 ━━━━━━━━━━━━━━━━━━━━ 63s 174ms/step - accuracy: 0.2681 - loss: 1.8347 - val_accuracy: 0.2803 - val_loss: 1.7847
Validation accuracy: 0.2628
Entrenando modelo: deeper_cnn, epochs: 20, batch_size: 32
Epoch 1/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 131s 172ms/step - accuracy: 0.2119 - loss: 4.3164 - val_accuracy: 0.2761 - val_loss: 1.9880
Epoch 2/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 130s 180ms/step - accuracy: 0.2679 - loss: 1.9237 - val_accuracy: 0.3665 - val_loss: 1.6534
Epoch 3/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 133s 184ms/step - accuracy: 0.3083 - loss: 1.7587 - val_accuracy: 0.2153 - val_loss: 1.8624
Validation accuracy: 0.2761
Entrenando modelo: deeper_cnn, epochs: 20, batch_size: 64
Epoch 1/20
361/361 ━━━━━━━━━━━━━━━━━━━━ 125s 330ms/step - accuracy: 0.2139 - loss: 4.5823 - val_accuracy: 0.2644 - val_loss: 2.3137
Epoch 2/20
361/361 ━━━━━━━━━━━━━━━━━━━━ 115s 318ms/step - accuracy: 0.2561 - loss: 2.1193 - val_accuracy: 0.3306 - val_loss: 1.7569
Epoch 3/20
361/361 ━━━━━━━━━━━━━━━━━━━━ 114s 316ms/step - accuracy: 0.2989 - loss: 1.7934 - val_accuracy: 0.3898 - val_loss: 1.6017
Validation accuracy: 0.2644
Entrenando modelo: deeper_cnn, epochs: 30, batch_size: 32
Epoch 1/30
721/721 ━━━━━━━━━━━━━━━━━━━━ 142s 181ms/step - accuracy: 0.2069 - loss: 4.2483 - val_accuracy: 0.2772 - val_loss: 1.9787
Epoch 2/30
721/721 ━━━━━━━━━━━━━━━━━━━━ 129s 178ms/step - accuracy: 0.2610 - loss: 1.9080 - val_accuracy: 0.3696 - val_loss: 1.6647
Epoch 3/30
721/721 ━━━━━━━━━━━━━━━━━━━━ 127s 176ms/step - accuracy: 0.3172 - loss: 1.7369 - val_accuracy: 0.4215 - val_loss: 1.5550
Validation accuracy: 0.2772
Entrenando modelo: deeper_cnn, epochs: 30, batch_size: 64
Epoch 1/30
361/361 ━━━━━━━━━━━━━━━━━━━━ 127s 335ms/step - accuracy: 0.2059 - loss: 4.6124 - val_accuracy: 0.2618 - val_loss: 2.3513
Epoch 2/30
361/361 ━━━━━━━━━━━━━━━━━━━━ 114s 316ms/step - accuracy: 0.2627 - loss: 2.1318 - val_accuracy: 0.3518 - val_loss: 1.7272
Epoch 3/30
361/361 ━━━━━━━━━━━━━━━━━━━━ 114s 316ms/step - accuracy: 0.3159 - loss: 1.7674 - val_accuracy: 0.3476 - val_loss: 1.8590
Validation accuracy: 0.2618
Entrenando modelo: deep_cnn, epochs: 20, batch_size: 32
Epoch 1/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 373s 509ms/step - accuracy: 0.2048 - loss: 6.1184 - val_accuracy: 0.2638 - val_loss: 2.0061
Epoch 2/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 366s 507ms/step - accuracy: 0.2615 - loss: 1.9297 - val_accuracy: 0.3110 - val_loss: 1.7986
Epoch 3/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 354s 491ms/step - accuracy: 0.3161 - loss: 1.7631 - val_accuracy: 0.3495 - val_loss: 1.6514
Validation accuracy: 0.2638
Entrenando modelo: deep_cnn, epochs: 20, batch_size: 64
Epoch 1/20
361/361 ━━━━━━━━━━━━━━━━━━━━ 329s 896ms/step - accuracy: 0.2088 - loss: 6.7566 - val_accuracy: 0.2598 - val_loss: 2.5125
Epoch 2/20
361/361 ━━━━━━━━━━━━━━━━━━━━ 315s 872ms/step - accuracy: 0.2583 - loss: 2.1152 - val_accuracy: 0.3008 - val_loss: 2.0187
Epoch 3/20
361/361 ━━━━━━━━━━━━━━━━━━━━ 314s 870ms/step - accuracy: 0.3224 - loss: 1.7686 - val_accuracy: 0.4045 - val_loss: 1.5829
Validation accuracy: 0.2598
Entrenando modelo: deep_cnn, epochs: 30, batch_size: 32
Epoch 1/30
721/721 ━━━━━━━━━━━━━━━━━━━━ 369s 501ms/step - accuracy: 0.2142 - loss: 6.1681 - val_accuracy: 0.2841 - val_loss: 1.9777
Epoch 2/30
721/721 ━━━━━━━━━━━━━━━━━━━━ 362s 502ms/step - accuracy: 0.2639 - loss: 1.9308 - val_accuracy: 0.3452 - val_loss: 1.6797
Epoch 3/30
721/721 ━━━━━━━━━━━━━━━━━━━━ 360s 499ms/step - accuracy: 0.3285 - loss: 1.7607 - val_accuracy: 0.3802 - val_loss: 1.6487
Validation accuracy: 0.2841
Entrenando modelo: deep_cnn, epochs: 30, batch_size: 64
Epoch 1/30
361/361 ━━━━━━━━━━━━━━━━━━━━ 356s 972ms/step - accuracy: 0.2174 - loss: 6.6183 - val_accuracy: 0.2369 - val_loss: 2.4241
Epoch 2/30
361/361 ━━━━━━━━━━━━━━━━━━━━ 355s 981ms/step - accuracy: 0.2796 - loss: 2.0770 - val_accuracy: 0.3450 - val_loss: 1.7224
Epoch 3/30
361/361 ━━━━━━━━━━━━━━━━━━━━ 349s 965ms/step - accuracy: 0.3412 - loss: 1.7565 - val_accuracy: 0.4179 - val_loss: 1.6044
Validation accuracy: 0.2369
Entrenando modelo: very_deep_cnn, epochs: 20, batch_size: 32
Epoch 1/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 464s 630ms/step - accuracy: 0.2179 - loss: 6.1639 - val_accuracy: 0.2905 - val_loss: 1.8682
Epoch 2/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 456s 633ms/step - accuracy: 0.2669 - loss: 1.8500 - val_accuracy: 0.2343 - val_loss: 1.8230
Epoch 3/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 458s 635ms/step - accuracy: 0.3335 - loss: 1.7199 - val_accuracy: 0.4158 - val_loss: 1.5330
Validation accuracy: 0.2905
Entrenando modelo: very_deep_cnn, epochs: 20, batch_size: 64
Epoch 1/20
361/361 ━━━━━━━━━━━━━━━━━━━━ 452s 1s/step - accuracy: 0.2174 - loss: 7.0579 - val_accuracy: 0.2616 - val_loss: 2.2115
Epoch 2/20
361/361 ━━━━━━━━━━━━━━━━━━━━ 451s 1s/step - accuracy: 0.2692 - loss: 1.9911 - val_accuracy: 0.3105 - val_loss: 1.8351
Epoch 3/20
361/361 ━━━━━━━━━━━━━━━━━━━━ 446s 1s/step - accuracy: 0.3327 - loss: 1.7162 - val_accuracy: 0.4317 - val_loss: 1.5291
Validation accuracy: 0.2616
Entrenando modelo: very_deep_cnn, epochs: 30, batch_size: 32
Epoch 1/30
721/721 ━━━━━━━━━━━━━━━━━━━━ 465s 634ms/step - accuracy: 0.2155 - loss: 6.2229 - val_accuracy: 0.2881 - val_loss: 1.8794
Epoch 2/30
721/721 ━━━━━━━━━━━━━━━━━━━━ 455s 630ms/step - accuracy: 0.2677 - loss: 1.8485 - val_accuracy: 0.2347 - val_loss: 1.8292
Epoch 3/30
721/721 ━━━━━━━━━━━━━━━━━━━━ 411s 570ms/step - accuracy: 0.3134 - loss: 1.7343 - val_accuracy: 0.3403 - val_loss: 1.6891
Validation accuracy: 0.2881
Entrenando modelo: very_deep_cnn, epochs: 30, batch_size: 64
Epoch 1/30
361/361 ━━━━━━━━━━━━━━━━━━━━ 422s 1s/step - accuracy: 0.2096 - loss: 6.9968 - val_accuracy: 0.1662 - val_loss: 2.2206
Epoch 2/30
361/361 ━━━━━━━━━━━━━━━━━━━━ 398s 1s/step - accuracy: 0.2655 - loss: 1.9979 - val_accuracy: 0.2425 - val_loss: 1.8405
Epoch 3/30
361/361 ━━━━━━━━━━━━━━━━━━━━ 399s 1s/step - accuracy: 0.3164 - loss: 1.7452 - val_accuracy: 0.4180 - val_loss: 1.5954
Validation accuracy: 0.1662

In [44]:
# Imprimir los mejores parámetros y el mejor score
print(f"Mejores Parámetros: {best_params}")
print(f"Mejor Score: {best_accuracy}")

Mejores Parámetros: {}
Mejor Score: 0


In [ ]:
# Cargar y procesar imágenes de prueba
test_images = load_and_preprocess_images2(test_data)

In [ ]:
# Realizar predicciones sobre el conjunto de prueba
best_model.load_weights('best_model.keras')  # Cargar los mejores pesos
predictions = best_model.predict(test_images)

In [ ]:
# Generar el archivo de submission con el nombre del modelo
submission_name = f"submission_{best_params['model_type']}.csv"
submission = sample_submission.copy()

In [ ]:
# Decodificar las etiquetas predichas
predicted_labels = np.argmax(predictions, axis=1)
predicted_labels_decoded = label_encoder.inverse_transform(predicted_labels)

In [ ]:
# Asegurar que el submission contenga las columnas para las 7 emociones
submission['label'] = predicted_labels_decoded

submission.to_csv(f'data/submission/{submission_name}', index=False)

print(f"Submission file saved as {submission_name}")